In [ ]:
import pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
def parte_linea(linea):
    datos = linea.split(';')
    if not datos[0].isdigit():
        return None
    return [f.strip() for f in datos]
def to_float(value):
    try:
        return float(value.replace(',', '.'))
    except (ValueError, AttributeError):
        return None
raw_data = sc.textFile("calidad_aire_datos_meteo_mes.csv").map(parte_linea).filter(lambda x: x is not None).cache()
#Ejercicio 1
def has_valid_hour(row):
    valid_indices = range(9, 56, 2)
    return any(row[i] == 'V' for i in valid_indices)

temp_rdd = raw_data.filter(lambda x: x[3] == '83')
valid_records_count = temp_rdd.filter(has_valid_hour).count()

print(f"Registros de temperatura con al menos una hora válida: {valid_records_count}")
print("-" * 30)

#Ejercicio 2

def get_daily_max(row):
    fecha = f"{row[5]}-{row[6].zfill(2)}-{row[7].zfill(2)}"
    valores = []
    for i in range(9, 56, 2):
        val = to_float(row[i-1])
        if row[i] == 'V' and val is not None:
            valores.append(val)
    
    return (fecha, max(valores)) if valores else None

max_temp_por_dia = temp_rdd.map(get_daily_max).filter(lambda x: x is not None).reduceByKey(max).sortByKey()

print(f"\nMáxima temperatura por día:")
for fecha, valor in max_temp_por_dia.collect():
    print(f"{fecha}: {valor} ºC")
print("-" * 30)

#Ejercicio 3

def get_precip_data(row):
    fecha = f"{row[5]}-{row[6].zfill(2)}-{row[7].zfill(2)}"
    municipio = row[1]
    estacion = row[2]
    suma_dia = sum(to_float(row[i-1]) for i in range(9, 56, 2) 
                   if row[i] == 'V' and to_float(row[i-1]) is not None)
    
    return (fecha, (municipio, estacion, suma_dia))

precip_rdd = raw_data.filter(lambda x: x[3] == '89').map(get_precip_data)

max_precip_estacion_dia = precip_rdd.reduceByKey(lambda a, b: a if a[2] >= b[2] else b).sortByKey()

print(f"\nMayor precipitación registrada cada día:")
for fecha, (muni, est, total) in max_precip_estacion_dia.collect():
    print(f"Fecha: {fecha} | Muni: {muni} | Est: {est} | Total: {total:.2f} mm")

abs_max = max_precip_estacion_dia.reduce(lambda a, b: a if a[1][2] >= b[1][2] else b)
print(f"\nMayor precipitación registrada:")
print(f"Fecha: {abs_max[0]} | Municipio: {abs_max[1][0]} | Estación: {abs_max[1][1]} | Valor: {abs_max[1][2]:.2f} mm")
print("-" * 30)

#Ejercicio 4
def get_avg_temp(row):
    fecha = f"{row[5]}-{row[6].zfill(2)}-{row[7].zfill(2)}"
    valores = [to_float(row[i-1]) for i in range(9, 56, 2) 
               if row[i] == 'V' and to_float(row[i-1]) is not None]
    
    if not valores: return None
    return (fecha, sum(valores) / len(valores))

ref_st = temp_rdd.filter(lambda x: x[1] == '6' and x[2] == '4').map(get_avg_temp).filter(lambda x: x)
comp_st = temp_rdd.filter(lambda x: x[1] == '5' and x[2] == '2').map(get_avg_temp).filter(lambda x: x)

comparativa = ref_st.join(comp_st)

def calcular_porcentaje(valores):
    media_ref, media_comp = valores
    if media_ref == 0: return 0.0
    return (media_comp / media_ref) * 100

resultados_finales = comparativa.mapValues(calcular_porcentaje).sortByKey()

print(f"\nComparativa(porcentaje) del valor de la estación de referencia:")
for fecha, porcentaje in resultados_finales.collect():
    print(f"{fecha} -> {porcentaje:.2f}%")

Registros de temperatura con al menos una hora válida: 224
------------------------------
2026-02-01: 11.7 ºC
2026-02-02: 12.2 ºC
2026-02-03: 9.8 ºC
2026-02-04: 11.3 ºC
2026-02-05: 15.5 ºC
2026-02-06: 10.3 ºC
2026-02-07: 9.0 ºC
2026-02-08: 12.7 ºC
------------------------------
Fecha: 2026-02-01 | Muni: 120 | Est: 1 | Total: 20.80 mm
Fecha: 2026-02-02 | Muni: 161 | Est: 1 | Total: 18.40 mm
Fecha: 2026-02-03 | Muni: 127 | Est: 4 | Total: 7.60 mm
Fecha: 2026-02-04 | Muni: 45 | Est: 2 | Total: 11.60 mm
Fecha: 2026-02-05 | Muni: 115 | Est: 3 | Total: 30.80 mm
Fecha: 2026-02-06 | Muni: 67 | Est: 1 | Total: 6.30 mm
Fecha: 2026-02-07 | Muni: 115 | Est: 3 | Total: 21.80 mm
Fecha: 2026-02-08 | Muni: 120 | Est: 1 | Total: 25.10 mm

MAYOR PRECIPITACIÓN REGISTRADA:
Fecha: 2026-02-05 | Municipio: 115 | Estación: 3 | Valor: 30.80 mm
------------------------------
2026-02-01 -> 129.04%
2026-02-02 -> 113.45%
2026-02-03 -> 114.98%
2026-02-04 -> 114.01%
2026-02-05 -> 112.09%
2026-02-06 -> 116.52%
2026-0